In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
palace2022 = pd.read_csv("./4대궁2022.csv", encoding='cp949')
df_palace2022 = palace2022[['일자', '경복궁(유료)', '경복궁(무료)', '덕수궁(유료)', '덕수궁(무료)', '창경궁(유료)', '창경궁(무료)', '창덕궁(유료)', '창덕궁(무료)']]

df_palace2022['경복궁'] =  df_palace2022['경복궁(유료)']+df_palace2022['경복궁(무료)']
df_palace2022['덕수궁'] =  df_palace2022['덕수궁(유료)']+df_palace2022['덕수궁(무료)']
df_palace2022['창경궁'] =  df_palace2022['창경궁(유료)']+df_palace2022['창경궁(무료)']
df_palace2022['창덕궁'] =  df_palace2022['창덕궁(유료)']+df_palace2022['창덕궁(무료)']


df_palace2022[['경복궁']].to_csv('경복궁2022.csv', index=True)
df_palace2022[['덕수궁']].to_csv('덕수궁2022.csv', index=True)
df_palace2022[['창경궁']].to_csv('창경궁2022.csv', index=True)
df_palace2022[['창덕궁']].to_csv('창덕궁2022.csv', index=True)


C:\Users\01062\AppData\Local\Temp/ipykernel_14388/2470930722.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_palace2022['경복궁'] =  df_palace2022['경복궁(유료)']+df_palace2022['경복궁(무료)']
C:\Users\01062\AppData\Local\Temp/ipykernel_14388/2470930722.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_palace2022['덕수궁'] =  df_palace2022['덕수궁(유료)']+df_palace2022['덕수궁(무료)']
C:\Users\01062\AppData\Local\Temp/ipykernel_14388/2470930722.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [5]:
palace2023 = pd.read_csv("./4대궁2023.csv", encoding='cp949')
df_palace2023 = palace2023[['일자', '경복궁(유료)', '경복궁(무료)', '덕수궁(유료)', '덕수궁(무료)', '창경궁(유료)', '창경궁(무료)', '창덕궁(유료)', '창덕궁(무료)']]

df_palace2023['경복궁'] =  df_palace2023['경복궁(유료)']+df_palace2023['경복궁(무료)']
df_palace2023['덕수궁'] =  df_palace2023['덕수궁(유료)']+df_palace2023['덕수궁(무료)']
df_palace2023['창경궁'] =  df_palace2023['창경궁(유료)']+df_palace2023['창경궁(무료)']
df_palace2023['창덕궁'] =  df_palace2023['창덕궁(유료)']+df_palace2023['창덕궁(무료)']

df_palace2022[['경복궁']].to_csv('경복궁2023.csv', index=True)
df_palace2022[['덕수궁']].to_csv('덕수궁2023.csv', index=True)
df_palace2022[['창경궁']].to_csv('창경궁2023.csv', index=True)
df_palace2022[['창덕궁']].to_csv('창덕궁2023.csv', index=True)


C:\Users\01062\AppData\Local\Temp/ipykernel_14388/12243983.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_palace2023['경복궁'] =  df_palace2023['경복궁(유료)']+df_palace2023['경복궁(무료)']
C:\Users\01062\AppData\Local\Temp/ipykernel_14388/12243983.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_palace2023['덕수궁'] =  df_palace2023['덕수궁(유료)']+df_palace2023['덕수궁(무료)']
C:\Users\01062\AppData\Local\Temp/ipykernel_14388/12243983.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [7]:
seoul_park_xls = pd.ExcelFile('./서울대공원2023.xlsx')
month = ['1월', '2월 ', '3월 ', '4월 ', '5월 ', '6월 ', '7월', '8월', '9월', '10월', '11월', '12월']

# 각 시트에서 특정 열만 가져와서 리스트에 저장
dfs = []  
for sheet in month:
  seoul_park = pd.read_excel(seoul_park_xls, sheet_name=sheet, header=4)
  seoul_park.drop(0, inplace=True)
  df = seoul_park[['날짜', '일합계']]  # 원하는 열만 가져오기
  dfs.append(df)

# 모든 시트의 특정 열을 세로로 연결
merged_df = pd.concat(dfs, ignore_index=True)
merged_df
# 새로운 엑셀 파일로 저장 (기존 파일 수정하려면 다른 방식 필요)
merged_df.to_csv("merged_seoul_park_2023.csv", index=False)


ImportError: Pandas requires version '3.1.0' or newer of 'openpyxl' (version '3.0.10' currently installed).

In [8]:
seoul_park_xls = pd.ExcelFile('./서울대공원2024.xlsx')
month = ['1월', '2월 ', '3월 ', '4월 ', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월']

# 각 시트에서 특정 열만 가져와서 리스트에 저장
dfs = []  
for sheet in month:
  seoul_park = pd.read_excel(seoul_park_xls, sheet_name=sheet, header=4)
  seoul_park.drop(0, inplace=True)
  df = seoul_park[['날짜', '일합계']]  # 원하는 열만 가져오기
  dfs.append(df)

# 모든 시트의 특정 열을 세로로 연결
merged_df = pd.concat(dfs, ignore_index=True)
merged_df
# 새로운 엑셀 파일로 저장 (기존 파일 수정하려면 다른 방식 필요)
merged_df.to_csv("merged_seoul_park_2024.csv", index=False)


ImportError: Pandas requires version '3.1.0' or newer of 'openpyxl' (version '3.0.10' currently installed).